In [2]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import base64
import plotly
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from Animal_Shelter import AnimalShelter

username = "aacuser"
password = "elf"
shelter = AnimalShelter(username, password)

# readAll is the method name from AnimalShelter class and returns a cursor object
# update the next line to match your method name from your CRUD Python module class
df = pd.DataFrame(list(shelter.read_all({"animal_type":"Dog"})))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'gsLogo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.Img(id='customer-image',src='data:image/png;base64,{}'.format(encoded_image.decode()),alt='customer image')),
    html.Center(html.H4('This is me, Larry Arndt for CS-340')),
    html.Hr(),
    html.Div([
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        dcc.RadioItems(id='filter-type',
        options = [{'label':'Water Rescue','value':'WR'},
                   {'label':'Mountain Rescue','value':'MR'},
                   {'label':'Disaster Tracking','value':'DT'},
                   {'label':'Reset','value':'R'},
        ],
        value = 'Reset'
        ),

   ]),
   html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        editable=False,
        #filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=True,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
             [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries
        if filter_type == 'WR':
            df = pd.DataFrame(list(shelter.read_all({"animal_type":"Dog","breed":{"$in":["Labrador Retriever Mix","Chesa Bay Retr Mix","Newfoundland Mix"]}, "sex_upon_outcome":"Intact Female","age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}})))
        elif filter_type == 'MR':
            df = pd.DataFrame(list(shelter.read_all({"animal_type":"Dog","breed":{"$in":["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky", "Rottweiler"]}, "sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}})))
        elif filter_type == 'DT':
            df = pd.DataFrame(list(shelter.read_all({"animal_type":"Dog","breed":{"$in":["Doberman Pinsch","German Shepherd","Golden Retriever","Bloodhound", "Rottweiler"]}, "sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gte":20},"age_upon_outcome_in_weeks":{"$lte":300}})))
        else:
            df = pd.DataFrame(list(shelter.read_all({"animal_type":"Dog"})))
            
            
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(
            
            figure = px.pie(
                     data_frame=dff,
                     values=dff.age_upon_outcome_in_weeks,
                     names=dff.breed)
        )    
    ]
    
    
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.iloc[0,13],dff.iloc[0,14]], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[0,9])
                ])
            ])
        ])
    ]


app